In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import json
import os

from dotenv import load_dotenv
load_dotenv()

# Google_API_KEY = os.getenv("g_key")
Google_API_KEY = os.getenv("g_key")

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vredendal,ZA,2022-08-01 04:44:56,-31.6683,18.5012,45.95,59,0,6.42
1,1,Matara,LK,2022-08-01 04:44:56,5.9485,80.5353,78.51,89,98,15.99
2,2,Bethel,US,2022-08-01 04:44:57,41.3712,-73.4140,74.25,88,75,3.44
3,3,Praia,CV,2022-08-01 04:44:58,14.9215,-23.5087,77.54,73,75,11.50
4,4,Vila,PT,2022-08-01 04:44:58,42.0304,-8.1588,63.48,51,0,4.36


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=os.environ["g_key"])

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]


# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig




Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Matara,LK,2022-08-01 04:44:56,5.9485,80.5353,78.51,89,98,15.99
3,3,Praia,CV,2022-08-01 04:44:58,14.9215,-23.5087,77.54,73,75,11.50
9,9,Kavieng,PG,2022-08-01 04:45:01,-2.5744,150.7967,85.06,71,82,17.45
10,10,Zabol,AF,2022-08-01 04:45:02,32.2500,67.2500,79.09,25,0,9.06
12,12,Yichun,CN,2022-08-01 04:45:04,47.7000,128.9000,86.41,49,17,8.86
14,14,Villarrobledo,ES,2022-08-01 04:45:05,39.2699,-2.6012,77.77,70,0,13.62
18,18,Taikang,CN,2022-08-01 04:45:07,46.8333,124.4167,88.92,52,0,9.46
19,19,Vaini,TO,2022-08-01 04:45:08,-21.2000,-175.2000,78.96,94,75,6.91
21,21,Atuona,PF,2022-08-01 04:45:09,-9.8000,-139.0333,77.25,74,13,11.41
25,25,Bengkulu,ID,2022-08-01 04:45:11,-3.8004,102.2655,84.49,68,54,6.69


In [9]:
preferred_cities_df.count()


City_ID       200
City          200
Country       200
Date          200
Lat           200
Lng           200
Max Temp      200
Humidity      200
Cloudiness    200
Wind Speed    200
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Matara,LK,78.51,5.9485,80.5353,
3,Praia,CV,77.54,14.9215,-23.5087,
9,Kavieng,PG,85.06,-2.5744,150.7967,
10,Zabol,AF,79.09,32.2500,67.2500,
12,Yichun,CN,86.41,47.7000,128.9000,
14,Villarrobledo,ES,77.77,39.2699,-2.6012,
18,Taikang,CN,88.92,46.8333,124.4167,
19,Vaini,TO,78.96,-21.2000,-175.2000,
21,Atuona,PF,77.25,-9.8000,-139.0333,
25,Bengkulu,ID,84.49,-3.8004,102.2655,


In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Matara,LK,78.51,5.9485,80.5353,
3,Praia,CV,77.54,14.9215,-23.5087,
9,Kavieng,PG,85.06,-2.5744,150.7967,
10,Zabol,AF,79.09,32.2500,67.2500,
12,Yichun,CN,86.41,47.7000,128.9000,
14,Villarrobledo,ES,77.77,39.2699,-2.6012,
18,Taikang,CN,88.92,46.8333,124.4167,
19,Vaini,TO,78.96,-21.2000,-175.2000,
21,Atuona,PF,77.25,-9.8000,-139.0333,
25,Bengkulu,ID,84.49,-3.8004,102.2655,


In [27]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_API_KEY
}

In [28]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Matara,LK,78.51,5.9485,80.5353,Amaloh Boutique Resort
3,Praia,CV,77.54,14.9215,-23.5087,LT Aparthotel
9,Kavieng,PG,85.06,-2.5744,150.7967,Nusa Island Retreat
10,Zabol,AF,79.09,32.2500,67.2500,
12,Yichun,CN,86.41,47.7000,128.9000,锦江酒店
14,Villarrobledo,ES,77.77,39.2699,-2.6012,Hotel Castillo
18,Taikang,CN,88.92,46.8333,124.4167,Jia Hotel
19,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
21,Atuona,PF,77.25,-9.8000,-139.0333,Villa Enata
25,Bengkulu,ID,84.49,-3.8004,102.2655,Grage Hotel Bengkulu


In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))